# Create sagemaker endpoints
This notebook creates sagemaker endpoints with the pretrained models

In [ ]:
import boto3
import sagemaker
import os
import numpy as np
import urllib.request

endpoint_instance_type = "ml.t2.medium"
sage_session = sagemaker.session.Session()
s3_bucket = sage_session.default_bucket()
role = sagemaker.get_execution_role()
print ()

### Make your own S3 bucket and copy the pre-trained models into your own bucket
Package the pretrained models

In [ ]:
!mv PretrainedModels Models
!tar -czf Models.tar.gz Models
!mv Models PretrainedModels

An s3 bucket is a cloud storage. In this section, we will make a bucket in your AWS account then copy the pre-trained models into your own bucket

In [ ]:
s3_client = boto3.client('s3')
s3_client.upload_file("Models.tar.gz", s3_bucket, 
                      "battlesnake-pretrainedmodels/Models.tar.gz")

# Create a sagemaker endpoint with the bucket


In [ ]:
map_size = (19, 19)
model_data = "s3://{}/battlesnake-pretrainedmodels/Models.tar.gz".format(s3_bucket)
print("Make an endpoint with {}".format(model_data))

In [ ]:
from sagemaker.mxnet import MXNetModel
mxnet_model = MXNetModel(model_data=model_data,
                             entry_point='predict.py',
                             role=role,
                             framework_version='1.6.0',
                             source_dir='SnakeInference',
                             name="battlesnake-mxnet",
                             py_version='py3')
predictor = mxnet_model.deploy(initial_instance_count=1,
                               instance_type="local", #endpoint_instance_type,
                               update_endpoint=False,
                               endpoint_name='battlesnake-endpoint')

# Test the model

In [ ]:
data1 = np.zeros(shape=(1, 2, 3, map_size[0]+2, map_size[1]+2))
data2 = np.zeros(shape=(1, 2))
data3 = np.zeros(shape=(1, 2))
data4 = np.zeros(shape=(1, 2))
health_dict = {0: 50, 1: 50}
json = {"board": {
            "height": 15,
            "width": 15,
            "food": [],
            "snakes": []
            },
        "you": {
            "id": "snake-id-string",
            "name": "Sneky Snek",
            "health": 90,
            "body": [{"x": 1, "y": 3}]
            }
        }
action = predictor.predict({"state": data1, "snake_id": data2, 
                           "turn_count": data3, "health": data4,  
                           "all_health": health_dict, "map_width": map_size[0], "json": json})
print("Action to take is {}".format(action))

# Clean up
Remove the packaged model

In [ ]:
!rm Models.tar.gz